In [38]:
#  DeBERTa Hierarchical Inference & Visualization Pipeline (Error-Free)

# STEP 1: Imports
import os, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch.nn.functional import softmax
from torch.utils.data import Dataset, DataLoader
from transformers import DebertaV2Tokenizer, DebertaV2ForSequenceClassification
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
from tqdm import tqdm

# STEP 2: Dataset Class
class CustomTextDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=256):
        self.encodings = tokenizer(texts, truncation=True, padding=True, max_length=max_len, return_tensors='pt')
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __getitem__(self, idx):
        return {
            'input_ids': self.encodings['input_ids'][idx],
            'attention_mask': self.encodings['attention_mask'][idx],
            'labels': self.labels[idx]
        }

    def __len__(self):
        return len(self.labels)

# STEP 3: Inference Function
def run_fold_inference(model_paths, tokenizer, texts, labels, num_labels, save_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    dataset = CustomTextDataset(texts, labels, tokenizer)
    loader = DataLoader(dataset, batch_size=32)
    all_probs = []

    for path in model_paths:
        print(f"Loading model: {path}")
        model = DebertaV2ForSequenceClassification.from_pretrained("microsoft/deberta-v3-small", num_labels=num_labels)
        state_dict = torch.load(path, map_location=device)
        w_shape = model.classifier.weight.shape
        b_shape = model.classifier.bias.shape
        state_dict = {k: v for k, v in state_dict.items()
                      if not (k == "classifier.weight" and v.shape != w_shape)
                      and not (k == "classifier.bias" and v.shape != b_shape)}
        model.load_state_dict(state_dict, strict=False)
        model.to(device).eval()

        probs = []
        with torch.no_grad():
            for batch in tqdm(loader, desc="Infer"):
                logits = model(input_ids=batch['input_ids'].to(device),
                               attention_mask=batch['attention_mask'].to(device)).logits
                probs.extend(softmax(logits, dim=1).cpu().numpy())

        all_probs.append(np.array(probs))

    probs = np.mean(np.stack(all_probs), axis=0)
    preds = np.argmax(probs, axis=1)

    os.makedirs(save_path, exist_ok=True)
    np.save(os.path.join(save_path, "preds.npy"), preds)
    np.save(os.path.join(save_path, "labels.npy"), labels)
    np.save(os.path.join(save_path, "probs.npy"), probs)
    return preds, probs

# STEP 4: Plotting Functions
def plot_confusion(y_true, y_pred, class_names, path):
    cm = confusion_matrix(y_true, y_pred, labels=list(range(len(class_names))))
    disp = ConfusionMatrixDisplay(cm, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(6, 6))
    disp.plot(cmap="Blues", ax=ax, colorbar=False)
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

def plot_metrics(y_true, y_pred, class_names, png_path, json_path):
    y_true = list(map(int, y_true))
    y_pred = list(map(int, y_pred))
    present = sorted(set(y_true) | set(y_pred))
    present_names = [class_names[i] for i in present]
    p, r, f, _ = precision_recall_fscore_support(y_true, y_pred, labels=present, average=None)
    p, r, f = np.array(p), np.array(r), np.array(f)

    metrics = {
        name: {"p": float(p_), "r": float(r_), "f": float(f_)}
        for name, p_, r_, f_ in zip(present_names, p, r, f)
    }
    with open(json_path, "w") as f_out:
        json.dump(metrics, f_out, indent=2)

    x = np.arange(len(p))
    plt.figure(figsize=(8, 5))
    plt.bar(x - 0.2, p, 0.2, label="Precision")
    plt.bar(x, r, 0.2, label="Recall")
    plt.bar(x + 0.2, f, 0.2, label="F1")
    plt.xticks(x, [str(n) for n in present_names], rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.savefig(png_path)
    plt.close()

def plot_confidence(probs, path):
    sns.histplot(np.max(probs, axis=1), bins=20)
    plt.title("Max Softmax Confidence")
    plt.tight_layout()
    plt.savefig(path)
    plt.close()

def plot_emb(probs, y_true, class_names, method, path):
    try:
        em = StandardScaler().fit_transform(np.array(probs))
        reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42) if method == "umap" \
            else TSNE(n_components=2, perplexity=30, random_state=42) if method == "tsne" \
            else PCA(n_components=2)
        reduced = reducer.fit_transform(em)
        y_true = list(map(int, list(y_true)))
        hue_labels = [class_names[y] if y < len(class_names) else str(y) for y in y_true]
        reduced = np.array(reduced)

        plt.figure(figsize=(7, 6))
        sns.scatterplot(x=reduced[:, 0], y=reduced[:, 1], hue=list(hue_labels), palette='Set2', s=60)
        plt.title(f"{method.upper()} Projection")
        plt.legend(loc='best', bbox_to_anchor=(1.05, 1))
        plt.tight_layout()
        plt.savefig(path)
        plt.close()

    except Exception as e:
        print(f"Failed to generate {method.upper()} plot → {path}: {e}")

# STEP 5: Subset Processor
def process_subset(df, level, cfg):
    df = df.dropna(subset=[cfg["label_col"]]).copy()
    texts = df["text"].tolist()
    labels = df[cfg["label_col"]].astype(int).tolist()
    preds, probs = run_fold_inference(cfg["model_paths"], tokenizer, texts, labels, cfg["num_labels"], cfg["run_path"])
    out_dir = os.path.join(cfg["run_path"], "plots", cfg["source"], level)
    os.makedirs(out_dir, exist_ok=True)
    plot_confusion(labels, preds, cfg["class_names"], f"{out_dir}/confusion.png")
    plot_metrics(labels, preds, cfg["class_names"], f"{out_dir}/metrics.png", f"{out_dir}/metrics.json")
    plot_confidence(probs, f"{out_dir}/confidence.png")
    for method in ["umap", "tsne", "pca"]:
        plot_emb(probs, labels, cfg["class_names"], method, f"{out_dir}/{method}.png")
    return pd.Series(preds, index=df.index)

# STEP 6: Config
LEVELS = {
    "level1": {
        "model_paths": [f"/content/drive/MyDrive/FIRE/run_20250628_034630/models/level1_fold{i}.pth" for i in range(1, 6)],
        "class_names": ["Noise", "Objective", "Subjective"],
        "num_labels": 3,
        "label_col": "level_1",
        "run_path": "/content/drive/MyDrive/FIRE/run_20250628_034630"
    },
    "level2": {
        "model_paths": [f"/content/drive/MyDrive/FIRE/run_20250629_090953/models/level2_fold{i}.pth" for i in range(1, 6)],
        "class_names": ["Neutral", "Negative", "Positive"],
        "num_labels": 3,
        "label_col": "level_2",
        "run_path": "/content/drive/MyDrive/FIRE/run_20250629_090953"
    },
    "level3": {
        "model_paths": [f"/content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold{i}.pth" for i in range(1, 6)],
        "class_names": ["Neutral Sentiments", "Questions", "Advertisements", "Miscellaneous"],
        "num_labels": 4,
        "label_col": "level_3",
        "run_path": "/content/drive/MyDrive/FIRE/run_20250629_121221"
    }
}

# STEP 7: Tokenizer & Data

tokenizer = DebertaV2Tokenizer.from_pretrained("microsoft/deberta-v3-small")
df = pd.read_csv("/content/drive/MyDrive/FIRE/crypto_task1_val.csv")
df["source"] = df["source"].str.lower()

# STEP 8: Hierarchical Inference
for src in ["twitter", "youtube", "reddit"]:
    df_src = df[df["source"] == src].copy()
    if df_src.empty:
        print(f"Skipping {src.upper()} (no data)")
        continue

    LEVELS["level1"]["source"] = src
    print(f"Running Level 1 on {src.upper()} ({len(df_src)})")
    df_src["L1p"] = process_subset(df_src, "level1", LEVELS["level1"])

    mask2 = df_src["L1p"] == 2
    df2 = df_src[mask2].copy().dropna(subset=["level_2"])
    if not df2.empty:
        LEVELS["level2"]["source"] = src
        print(f"Running Level 2 on {src.upper()} → {len(df2)} subjective posts")
        df2["L2p"] = process_subset(df2, "level2", LEVELS["level2"])
        df_src.loc[df2.index, "L2p"] = df2["L2p"]

        mask3 = (df_src["L1p"] == 2) & (df_src["L2p"] == 0)
        df3 = df_src[mask3].copy().dropna(subset=["level_3"])
        if not df3.empty:
            LEVELS["level3"]["source"] = src
            print(f"Running Level 3 on {src.upper()} → {len(df3)} neutral-subjective posts")
            df3["L3p"] = process_subset(df3, "level3", LEVELS["level3"])
            df_src.loc[df3.index, "L3p"] = df3["L3p"]

    df_src.to_csv(f"{LEVELS['level1']['run_path']}/plots/{src}/predictions.csv", index=False)
    print(f"Saved predictions for {src.upper()}")


Running Level 1 on TWITTER (429)
Loading model: /content/drive/MyDrive/FIRE/run_20250628_034630/models/level1_fold1.pth


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Infer: 100%|██████████| 14/14 [00:01<00:00,  7.97it/s]


Loading model: /content/drive/MyDrive/FIRE/run_20250628_034630/models/level1_fold2.pth


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Infer: 100%|██████████| 14/14 [00:01<00:00,  8.07it/s]


Loading model: /content/drive/MyDrive/FIRE/run_20250628_034630/models/level1_fold3.pth


...

Loading model: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold4.pth


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Infer: 100%|██████████| 9/9 [00:02<00:00,  3.65it/s]


Loading model: /content/drive/MyDrive/FIRE/run_20250629_121221/models/level3_fold5.pth


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Infer: 100%|██████████| 9/9 [00:02<00:00,  3.72it/s]
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Saved predictions for REDDIT
